In [247]:
# Chuyển mô hình sang device (GPU hoặc CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_size=wordEmbeddings.shape[0] 
word_emb_dim=wordEmbeddings.shape[1] 
word_embeddings=wordEmbeddings
case_emb_dim=len(case2Idx)
case_embeddings=caseEmbeddings 
char_size=len(char2Idx)
char_emb_dim=30
conv_out_channels=30
conv_kernel_size=3
lstm_hidden_size=200 
num_labels=len(label2Idx)
dropout=0.2
dropout_recurrent=0.2

# Khởi tạo mô hình (sử dụng dữ liệu embedding đã cho)
model = CNN_BLSTM(
    vocab_size=vocab_size, 
    word_emb_dim=word_emb_dim, 
    word_embeddings=word_embeddings, 
    case_emb_dim=case_emb_dim,
    case_embeddings=case_embeddings, 
    char_size=char_size, 
    char_emb_dim=char_emb_dim, 
    conv_out_channels=conv_out_channels, 
    conv_kernel_size=conv_kernel_size, 
    lstm_hidden_size=lstm_hidden_size, 
    num_labels=num_labels,
    dropout=dropout, 
    dropout_recurrent=dropout_recurrent,
    device=device
)
model.to(device)




# Khởi tạo Optimizer và Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # Loss cho classification

# Huấn luyện mô hình
train_model(model, train_loaders, dev_loaders, optimizer, criterion, epochs=30)
# train_model_baseline(model, train_loaders, dev_loaders, optimizer, criterion, epochs=30)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/30, Loss: 0.3413
Train Precision: 0.8258, Recall: 0.8041, F1: 0.8065
Dev Precision: 0.8230, Recall: 0.7965, F1: 0.8018

Epoch 2/30, Loss: 0.1315
Train Precision: 0.8270, Recall: 0.8546, F1: 0.8352
Dev Precision: 0.8157, Recall: 0.8397, F1: 0.8218

Epoch 3/30, Loss: 0.0983
Train Precision: 0.8651, Recall: 0.8651, F1: 0.8572
Dev Precision: 0.8444, Recall: 0.8476, F1: 0.8368

Epoch 4/30, Loss: 0.0793
Train Precision: 0.9164, Recall: 0.9012, F1: 0.9066
Dev Precision: 0.8941, Recall: 0.8817, F1: 0.8860

Epoch 5/30, Loss: 0.0656
Train Precision: 0.9248, Recall: 0.9078, F1: 0.9136
Dev Precision: 0.9033, Recall: 0.8821, F1: 0.8898

Epoch 6/30, Loss: 0.0528
Train Precision: 0.9385, Recall: 0.9346, F1: 0.9358
Dev Precision: 0.9076, Recall: 0.8996, F1: 0.9024

Epoch 7/30, Loss: 0.0432
Train Precision: 0.9462, Recall: 0.9485, F1: 0.9470
Dev Precision: 0.9096, Recall: 0.9107, F1: 0.9097

Epoch 8/30, Loss: 0.0374
Train Precision: 0.9573, Recall: 0.9558, F1: 0.9562
Dev Precision: 0.9176, Reca

In [248]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []
    
with torch.no_grad():
    for loader in test_loaders:
        for batch in loader:
            tokens, cases, chars, labels = batch
            tokens = tokens.to(model.device)
            cases  = cases.to(model.device)
            chars  = chars.to(model.device)
            labels = labels.to(model.device)
                
            outputs = model.forward(tokens, cases, chars)  # [batch_size, seq_len, num_labels]
            preds = torch.argmax(outputs, dim=-1)  # [batch_size, seq_len]
                
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    
y_pred = []
y_true = []
for pred_seq, true_seq in zip(all_preds, all_labels):
    for p, t in zip(pred_seq, true_seq):
            y_pred.append(p)
            y_true.append(t)
labels = list(label2Idx.values())
target_names = list(label2Idx.keys())
report = classification_report(y_true, y_pred, labels=labels, target_names=target_names, zero_division=0)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.91      0.93      0.92      1668
      B-MISC       0.81      0.83      0.82       702
       B-ORG       0.88      0.89      0.89      1661
       B-PER       0.96      0.94      0.95      1617
       I-LOC       0.77      0.91      0.84       257
      I-MISC       0.60      0.70      0.65       216
       I-ORG       0.85      0.86      0.86       835
       I-PER       0.97      0.99      0.98      1156
           O       1.00      0.99      0.99     38323

    accuracy                           0.98     46435
   macro avg       0.86      0.89      0.88     46435
weighted avg       0.98      0.98      0.98     46435

